# Computer Vision: Image Analysis

Images: 
- ![clear apples](https://www.kew.org/sites/default/files/styles/social/public/2022-05/apple%20cultivars.png?h=1d19dfc9&itok=NT2TwltK)
- ![tagliere](https://media.post.rvohealth.io/wp-content/uploads/2020/09/health-benefits-of-apples-1200x628-facebook-1200x628.jpg)

In [2]:
!wget -O apples.png https://www.kew.org/sites/default/files/styles/social/public/2022-05/apple%20cultivars.png?h=1d19dfc9&itok=NT2TwltK

--2023-11-17 20:18:43--  https://img.freepik.com/premium-photo/apple-fruit-basket-wooden-table-ripe-red-apples_73523-7545.jpg
Resolving img.freepik.com (img.freepik.com)... 23.38.194.45, 23.38.194.40, 2600:1404:ec00:e::17da:e85a, ...
Connecting to img.freepik.com (img.freepik.com)|23.38.194.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79745 (78K) [image/jpeg]
Saving to: ‘apples.png’

apples.png          100%[===================>]  77.88K   244KB/s    in 0.3s    

2023-11-17 20:18:45 (244 KB/s) - ‘apples.png’ saved [79745/79745]



In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load the image of apples
image = cv2.imread('apples.png')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (7, 7), 0)

# Perform edge detection
edges = cv2.Canny(blurred, 10, 100)

# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initialize a counter for the number of apples
apple_count = 0

# Iterate through the contours and filter out small ones (potential noise)
for contour in contours:
    if cv2.contourArea(contour) > 100:
        apple_count += 1

# Print the number of apples found
print(f"Number of apples: {apple_count}")

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw
import requests
from io import BytesIO

# Load the pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Modify the transformation for input images
transform = transforms.Compose([
    transforms.ToTensor()
])

# Function to perform object detection on an image
def detect_objects(model, image, threshold=0.5):
    # Transform the input image
    input_image = transform(image).unsqueeze(0)

    # Perform inference
    with torch.no_grad():
        prediction = model(input_image)

    # Filter out predictions below the threshold
    boxes = prediction[0]['boxes'][prediction[0]['scores'] > threshold]
    scores = prediction[0]['scores'][prediction[0]['scores'] > threshold]

    return boxes, scores

# Function to draw bounding boxes on the image
def draw_boxes(image, boxes, scores):
    draw = ImageDraw.Draw(image)
    for box in boxes:
        draw.rectangle([box[0], box[1], box[2], box[3]], outline="red", width=3)
    return image

# Function to visualize the detection results
def visualize_results(image, boxes, scores):
    image_with_boxes = draw_boxes(image, boxes, scores)
    display(image_with_boxes)

# Example image path
image_path = "apples.png"

# Load image and perform object detection
image = Image.open(image_path)

boxes, scores = detect_objects(model, image, threshold=0.5)

# Visualize the results
visualize_results(image, boxes, scores)